In [20]:
# Import libraries

import pandas as pd
import nltk
import numpy as np
from nltk.tokenize import PunktSentenceTokenizer,RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances, cosine_similarity
import warnings
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import text
import re

In [21]:
#to ignore deprecation warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [22]:
df = pd.read_csv("./processed_country.csv", encoding='latin-1')

In [23]:
df.head()

,name,continent,review,terrain,Zones
0,armenia,europe,"It might be a small nation, but Armenia is bi...",Armenian Highland with mountains; little fores...,Non listed third country
1,czechrepublic,europe,"A historic jewel at the heart of Europe, the ...",Bohemia in the west consists of rolling plains...,EU
2,chad,africa,From the natural wonders of the Sahara Desert ...,"broad, arid plains in center, desert in north,...",Non listed third country
3,cyprus,europe,"The legendary birthplace of Aphrodite, Cyprus ...",central plain with mountains to north and sout...,EU
4,ecuador,south-america,"Tiny by South American standards, Ecuador is ...","coastal plain (costa), inter-Andean central hi...",Non listed third country


In [28]:
df["review"].apply(lambda x: " ".join(re.findall("[a-zA-Z]*", x)).lower())

0       it  might  be  a  small  nation   but  armeni...
1       a  historic  jewel  at  the  heart  of  europ...
2      from  the  natural  wonders  of  the  sahara  ...
3      the  legendary  birthplace  of  aphrodite   cy...
4       tiny  by  south  american  standards   ecuado...
                             ...                        
192     if  it  were  an  authentic  taste  of  arabi...
193    vast  lakes  and  wetlands   long  and  life  ...
194    after  a  difficult  decade  or  so   stabilit...
195    northern  ireland  distils  the  best  of  bri...
196    guam  is  a  vibrant   tropical  paradise   co...
Name: review, Length: 197, dtype: object

In [29]:
# tokenize
tokenizer = RegexpTokenizer(r'\w+')
df["tokenized_text"] = df["review"].apply(lambda row: tokenizer.tokenize(row))

In [30]:
# lemmatize 
wnl = nltk.WordNetLemmatizer()

In [31]:
def lem(lst):
    list1=list()
    for i in lst : 
        list1.append(wnl.lemmatize(i))
    return list1

In [32]:
df["lemmatized_text"]=df["tokenized_text"].apply(lambda x : lem(x))

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer

my_additional_stop_words = ['acute', 'good', 'great', 'really', 'just', 'nice', 
                            'like', 'day', 'beautiful', 'visit', 'time', 'don',
                            'did', 'place', 'didn', 'did', 'tour', 'sydney','pm', 'the',
                            'lot', '00', 'inside', 'doesn','going','mostly', 'origin',
                            'right', '15']
stop_words = text.ENGLISH_STOP_WORDS.union(my_additional_stop_words)


tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words, strip_accents='unicode', norm='l2',lowercase=True)

In [37]:
X=[" ".join(text) for text in df["lemmatized_text"].values]
tfidf_matrix=tfidf_vectorizer.fit_transform(X)

In [38]:
tfidf_matrix.shape

(197, 8195)

In [39]:
similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [40]:
# Create a pandas series with movie titles as indices and indices as series values 
indices = pd.Series(df.index, index=df['name']).drop_duplicates()

In [41]:
indices

name
armenia              0
czechrepublic        1
chad                 2
cyprus               3
ecuador              4
                  ... 
yemen              192
zambia             193
zimbabwe           194
northernireland    195
guam               196
Length: 197, dtype: int64

In [51]:
title='thailand'

# Get the index corresponding to movie title
index = indices[title]

# Get the cosine similarity scores 
similarity_scores = list(enumerate(similarity_matrix[index]))


# Sort the similarity scores in descending order
sorted_similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)


# Top-10 most similar movie scores
top_10_movies_scores = sorted_similarity_scores[1:11]

In [52]:
# Get movie indices
top_10_movie_indices=[]
for i in top_10_movies_scores:
    top_10_movie_indices.append(i[0])
    
# Top 10 recommende movie
df['name'].iloc[top_10_movie_indices]

95            laos
36        cambodia
107       malaysia
139    philippines
121        myanmar
80       indonesia
108       maldives
68           ghana
41            fiji
56        dominica
Name: name, dtype: object